In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [2]:
test = pd.read_csv("test.csv").fillna('missing')
train = pd.read_csv("train.csv").fillna('missing')

In [3]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
test.head(20)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [5]:
train_text = train['comment_text']
test_text = test['comment_text']
concat_text = pd.concat([train_text, test_text]).str.lower()
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [6]:
def cleanData(text):
    txt = str(text)
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"n ig ger", "nigger", text)
    text = re.sub(r"s hit", "shit", text)
    text = re.sub(r"g ay", "gay", text)
    text = re.sub(r"f ag got", "faggot", text)
    text = re.sub(r"c ock", "cock", text)
    text = re.sub(r"cu nt", "cunt", text)
    text = re.sub(r"idi ot", "idiot", text)
    text = re.sub(r"(f|ht)tp(s?)://\\S+", " ", text)
    text = re.sub(r"http\\S+", "", text)
    text = re.sub(r"xml\\S+", "", text)
    text = re.sub(r"(a|e)w+\\b", "", text)
    text = re.sub(r"(y)a+\\b", "", text)
    text = re.sub(r"(w)w+\\b", "", text)
    text = re.sub(r"a?(ha)+\\b", "", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"i`m", "i am", text)
    text = re.sub(r"it`s", "it is", text)
    text = re.sub(r"i`ve", "i have", text)
    text = re.sub(r"you`re", "you are", text)
    text = re.sub(r"i`ve", "i have", text)
    text = re.sub(r"didn`t", "did not", text)
    text = re.sub(r"isn`t", "is not", text)
    text = re.sub(r"we`re", "we are", text)
    text = re.sub(r"you`ll", "you will", text)
    text = re.sub(r"they`re", "they are", text)
    text = re.sub(r"shouldn`t", "should not", text)
    text = re.sub(r"he`s", "he has", text)
    text = re.sub(r"wasn`t", "was not", text)
    text = re.sub(r"won`t", "will not", text)
    text = re.sub(r"wasn`t", "was not", text)
    text = re.sub(r"haven`t", "have not", text)
    text = re.sub(r"aren`t", "are not", text)
    text = re.sub(r"f4ck", "fuck", text)
    text = re.sub(r"fu", "fuck you", text)
    text = re.sub(r"fuckk", "fuck", text)
    text = re.sub(r"fucka", "fuck", text)
    text = re.sub(r"fuuck", "fuck", text)
    text = re.sub(r"fucck", "fuck", text)
    text = re.sub(r"fuc'k", "fuck", text)
    text = re.sub(r"fuckn", "fuck", text)
    text = re.sub(r"f'uck", "fuck", text)
    text = re.sub(r"fuckz", "fuck", text)
    text = re.sub(r"fyuck", "fuck", text)
    text = re.sub(r"fuck”", "fuck", text)
    text = re.sub(r"fuckkk", "fuck", text)
    text = re.sub(r"defuck", "fuck", text)
    text = re.sub(r"fuck43", "fuck", text)
    text = re.sub(r"nefuck", "fuck", text)
    text = re.sub(r"fuckng", "fuck", text)
    text = re.sub(r"fuckfu", "fuck", text)
    text = re.sub(r"fucks", "fuck", text)
    text = re.sub(r"fuicky", "fuck", text)
    text = re.sub(r"fuckum", "fuck", text)
    text = re.sub(r"fuckee", "fuck", text)
    text = re.sub(r"niggaz", "niggers", text)
    text = re.sub(r"niggerz", "niggers", text)
    text = re.sub(r"nigers", "niggers", text)
    text = re.sub(r"niger", "nigger", text)
    text = re.sub(r"niga", "nigger", text)
    text = re.sub(r"nig", "nigger", text)
    text = re.sub(r"nigor", "nigger", text)
    text = re.sub(r"nigra", "nigger", text)
    text = re.sub(r"nigre", "nigger", text)
    text = re.sub(r"nigar", "nigger", text)
    text = re.sub(r"niggor", "nigger", text)
    text = re.sub(r"niggur", "nigger", text)
    text = re.sub(r"nigga", "nigger", text)
    text = re.sub(r"niggah", "nigger", text)
    text = re.sub(r"niggar", "nigger", text)
    text = re.sub(r"niggah", "nigger", text)
    text = re.sub(r"niggress", "nigger", text)
    text = re.sub(r"fag", "faggot", text)
    text = re.sub(r"fagot", "faggot", text)
    text = re.sub(r"fagt", "faggot", text)
    text = re.sub(r"fagat", "faggot", text)
    text = re.sub(r"f4gg0t", "faggot", text)
    text = re.sub(r"fagg", "faggot", text)
    text = re.sub(r"faggs", "faggots", text)
    text = re.sub(r"fags", "faggots", text)

    return txt

In [7]:
concat_text = concat_text.map(lambda x: cleanData(x))

In [8]:
tv = TfidfVectorizer(sublinear_tf=True, 
                     strip_accents='unicode', 
                     analyzer='word', 
                     token_pattern=r'\w{1,}', 
                     ngram_range=(1, 1), 
                     stop_words='english', 
                     max_features=None)

In [9]:
tv.fit(concat_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [10]:
train_text_features = tv.transform(train_text)
test_text_features = tv.transform(test_text)

In [11]:
ctv = TfidfVectorizer(ngram_range=(1,5), 
                      strip_accents='unicode', 
                      use_idf=1, 
                      smooth_idf=1, 
                      sublinear_tf=1, 
                      analyzer='char', 
                      max_features=50000)

In [12]:
ctv.fit(concat_text)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=1,
        vocabulary=None)

In [13]:
train_symb_features = ctv.transform(train_text)
test_symb_features = ctv.transform(test_text)

In [14]:
X_train = hstack([train_text_features, train_symb_features])
X_test = hstack([test_text_features, test_symb_features])

In [15]:
scores = []
predictions = {'id': test['id']}
for class_name in class_names:
    y_train = train[class_name]
    classifier = LogisticRegression(solver='sag')
    classifier.fit(X_train, y_train)
    
    cv_score = np.mean(cross_val_score(classifier, X_train, y_train, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))
    
    predictions[class_name] = classifier.predict_proba(X_test)[:, 1]

CV score for class toxic is 0.9791552305267389
CV score for class severe_toxic is 0.9889143755861186
CV score for class obscene is 0.9906488972577057
CV score for class threat is 0.9894106458567586
CV score for class insult is 0.9832730488920998
CV score for class identity_hate is 0.9831076323725814


In [16]:
np.mean(scores)

0.98575163841533386

In [17]:
submission = pd.DataFrame.from_dict(predictions)
submission.to_csv('submission2.csv', index=False)